# Baseline TREC-50: TEXT Classification + BERT + Ax

## Librairies

In [ ]:
# !pip install transformers==4.8.2
# !pip install datasets==1.7.0
# !pip install ax-platform==0.1.20

In [ ]:
import os
import sys

In [ ]:
import io
import re
import pickle
from timeit import default_timer as timer

import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

from datasets import load_dataset, Dataset, concatenate_datasets
from transformers import AutoTokenizer
from transformers import BertModel
from transformers.data.data_collator import DataCollatorWithPadding

from ax import optimize
from ax.plot.contour import plot_contour
from ax.plot.trace import optimization_trace_single_method
from ax.service.managed_loop import optimize
from ax.utils.notebook.plotting import render, init_notebook_plotting

import esntorch.core.reservoir as res
import esntorch.core.learning_algo as la
import esntorch.core.merging_strategy as ms
import esntorch.core.baseline as bs

In [ ]:
%config Completer.use_jedi = False
%load_ext autoreload
%autoreload 2

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
SEED = 42

## Global variables

In [ ]:
RESULTS_PATH = '~/Results/Ax_results/Baseline' # path of your result folder
CACHE_DIR = '~/Data/huggignface/'              # path of your  folder

PARAMS_FILE = 'trec-50_baseline_params.pkl'
RESULTS_FILE = 'trec-50_baseline_results.pkl'

## Dataset

In [ ]:
# rename correct column as 'labels': depends on the dataset you load

def load_and_enrich_dataset(dataset_name, split, cache_dir):
    
    dataset = load_dataset(dataset_name, split=split, cache_dir=CACHE_DIR)
    
    dataset = dataset.rename_column('label-fine', 'labels') # 'label-coarse'
    dataset = dataset.map(lambda e: tokenizer(e['text'], truncation=True, padding=False), batched=True)
    dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

    def add_lengths(sample):
        sample["lengths"] = sum(sample["input_ids"] != 0)
        return sample
    
    dataset = dataset.map(add_lengths, batched=False)
    
    return dataset

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

full_train_dataset = load_and_enrich_dataset('trec', split='train', cache_dir=CACHE_DIR).sort("lengths")
train_val_datasets = full_train_dataset.train_test_split(train_size=0.8, shuffle=True)
train_dataset = train_val_datasets['train'].sort("lengths")
val_dataset = train_val_datasets['test'].sort("lengths")

test_dataset = load_and_enrich_dataset('trec', split='test', cache_dir=CACHE_DIR).sort("lengths")

dataset_d = {
    'full_train': full_train_dataset,
    'train': train_dataset,
    'val': val_dataset,
    'test': test_dataset
    }

dataloader_d = {}
for k, v in dataset_d.items():
    dataloader_d[k] = torch.utils.data.DataLoader(v, batch_size=256, collate_fn=DataCollatorWithPadding(tokenizer))

In [ ]:
dataset_d

## Optimization

In [ ]:
def fitness(alpha, 
            dataset_d, 
            dataloader_d, 
            return_test_acc=False):
    
    # parameters
    esn_params = {
                'embedding': 'bert-base-uncased', # TEXT.vocab.vectors,
                'input_dim' : 768,                        # dim of encoding!
                'learning_algo' : None,
                'criterion' : None,
                'optimizer' : None,
                'merging_strategy' : 'mean',
                'lexicon' : None,
                'bidirectional' : False,
                'device' : device,
                'seed' : 42
                 }

    # model
    ESN = bs.Baseline(**esn_params)

    ESN.learning_algo = la.RidgeRegression(alpha = alpha)# , mode='normalize')

    ESN = ESN.to(device)

    # predict
    if return_test_acc:
        t0 = timer()
        LOSS = ESN.fit(dataloader_d["train"])
        t1 = timer()
        acc = ESN.predict(dataloader_d["test"], verbose=False)[1].item()
    else:
        LOSS = ESN.fit(dataloader_d["train"])
        acc = ESN.predict(dataloader_d["val"], verbose=False)[1].item()

    # clean objects
    del ESN.learning_algo
    del ESN.criterion
    del ESN.merging_strategy
    del ESN
    torch.cuda.empty_cache()
    
    if return_test_acc:
        return acc, t1 - t0 
    else:
        return acc

In [ ]:
# # %%time

# fitness(alpha=10, dataset_d=dataset_d, dataloader_d=dataloader_d)

In [ ]:
def wrapped_fitness(d, return_test_acc=False):
    
    return fitness(alpha=d['alpha'],
                   dataset_d=dataset_d,
                   dataloader_d=dataloader_d,
                   return_test_acc=return_test_acc)

In [ ]:
best_params_d = {}

best_parameters, best_values, experiment, model = optimize(
        parameters=[
          {
            "name": "alpha",
            "value_type": "float",
            "type": "range",
            "log_scale": True,
            "bounds": [1e-3, 1e3],
          }
        ],
        # Booth function
        evaluation_function = wrapped_fitness,
        minimize = False,
        objective_name = 'val_accuracy',
        total_trials = 10
    )

# results
best_params_d['best_parameters'] = best_parameters
best_params_d['best_values'] = best_values
best_params_d['experiment'] = experiment
# best_params_d[res_dim]['model'] = model

## Results

In [ ]:
# best parameters

with open(os.path.join(RESULTS_PATH, PARAMS_FILE), 'wb') as fh:
    pickle.dump(best_params_d, fh)

In [ ]:
# # load results
# with open(os.path.join(RESULTS_PATH, PARAMS_FILE), 'rb') as fh:
#     best_params_d = pickle.load(fh)

In [ ]:
best_params_d

In [ ]:
# results

best_parameters = best_params_d['best_parameters']
acc, time = wrapped_fitness(best_parameters, return_test_acc=True)
results_tuple = acc, time
print("Experiment finished.")

In [ ]:
results_tuple

In [ ]:
with open(os.path.join(RESULTS_PATH, RESULTS_FILE), 'wb') as fh:
    pickle.dump(results_tuple, fh)